<a href="https://colab.research.google.com/github/vpvinc/P7_loan_scoring_model/blob/main/P7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

**P7 Implémentez un modèle de scoring**

https://www.kaggle.com/c/home-credit-default-risk/data

### TF Setup

In [1]:
# List all CPU and GPUs
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2919528543794474743]

### Seed and import libraries

In [2]:
"""répertoire courant où se trouve les données : D:/Dowloads_D/Data_OC/P7/Projet+Mise+en+prod+-+home-credit-default-risk"""
 
seed = 7
 # common libraries
import numpy as np
import pandas as pd
import os
import re
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
plt.rc('figure', figsize=(10, 6))
# extra libraries
import pickle
import IPython.display
from math import ceil
from operator import itemgetter

# uploading useful_functions.py from
# C:\Users\VP\Google Drive\Education\OC\working_directory
from google.colab import files
uploaded = files.upload()

Saving useful_functions.py to useful_functions.py


In [14]:
os.chdir('/content/')
%run -i useful_functions


# plot params
- set_style_pers

# EDA functions
 ## multiple dfs exploration
- dfs_insight
 ## univariate analysis
- plot_cont_kde
- plot_cat_countplot
# Clustering functions
- radar_ploting_clustering



### DL data from kaggle

In [5]:
## dowload datasets directly from kaggle to GD
# Import OS for navigation and environment set up
import os
# Check current location, '/content' is the Colab virtual machine
# os.getcwd()
# Enable the Kaggle environment, use the path to the directory your Kaggle API JSON is stored in
!pip install kaggle
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/Colab/Kaggle'
# Navigate into Drive where you want to store your Kaggle data
path_datasets = '/content/drive/MyDrive/Colab/Kaggle/Competitions'                                                 # /!\ variable # folder Competition / Datasets
os.chdir(path_datasets)
# Paste and run the copied API command, the data will download to the current directory
!kaggle competitions download -c home-credit-default-risk # replace dataset name/competition after -c              # /!\ variable # dataset
# Check contents of directory, you should see the .zip file for the competition in your Drive
# os.listdir()

## unzip datasets from GD to virtual machine 'content'
for file in os.listdir(path_datasets):
  # Complete path to storage location of the .zip file of data
  zip_path = os.path.join(path_datasets, file)
  # we create a folder 'data' in content that will host our unziped datasets and make it the WD
  if not os.path.exists('/content/sample_data/data'):
    os.mkdir('/content/sample_data/data')
  os.chdir('/content/sample_data/data') 
  # Copy the .zip file into the present directory
  !cp '{zip_path}' .
# once all ziped are copied in conten/sample data/data, we unzip them to CSV
!unzip \*.zip
# and delete the zip files
!rm *.zip
# check the content
# os.listdir()

bureau.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
application_test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
credit_card_balance.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
HomeCredit_columns_description.csv: Skipping, found more recently modified local copy (use --force to force download)
installments_payments.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
bureau_balance.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
POS_CASH_balance.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
previous_application.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
application_train.csv

# Shape Analysis

In [10]:
os.chdir('/content/sample_data/data')
!ls

application_test.csv	 HomeCredit_columns_description.csv
application_train.csv	 installments_payments.csv
bureau_balance.csv	 POS_CASH_balance.csv
bureau.csv		 previous_application.csv
credit_card_balance.csv  sample_submission.csv


In [11]:
os.chdir('/content/sample_data/data')
df_raw_app_test = pd.read_csv('application_test.csv')
df_raw_app_train = pd.read_csv('application_train.csv')
df_raw_bureau_balance = pd.read_csv('bureau_balance.csv')
df_raw_bureau = pd.read_csv('bureau.csv')
df_raw_credit_card = pd.read_csv('credit_card_balance.csv')
df_raw_installments = pd.read_csv('installments_payments.csv')
df_raw_POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv')
df_raw_previous_application = pd.read_csv('previous_application.csv')
df_raw_sample_submission = pd.read_csv('sample_submission.csv')

In [15]:
dfs_dict = dict()

dfs_dict['df_raw_app_test'] = df_raw_app_test
dfs_dict['df_raw_app_train'] = df_raw_app_train
dfs_dict['df_raw_bureau_balance'] = df_raw_bureau_balance
dfs_dict['df_raw_bureau'] = df_raw_bureau
dfs_dict['df_raw_credit_card'] = df_raw_credit_card
dfs_dict['df_raw_installments'] = df_raw_installments
dfs_dict['df_raw_POS_CASH_balance'] = df_raw_POS_CASH_balance
dfs_dict['df_raw_previous_application'] = df_raw_previous_application
dfs_dict['df_raw_sample_submission'] = df_raw_sample_submission

In [16]:
dfs_insight(dfs_dict)

  df-wise information 
                       name_df             shape   % NaN # duplicated rows
0              df_raw_app_test   ((48744, 121),)  23.81%                 0
1             df_raw_app_train  ((307511, 122),)  24.40%                 0
2        df_raw_bureau_balance  ((27299925, 3),)   0.00%                 0
3                df_raw_bureau  ((1716428, 17),)  13.50%                 0
4           df_raw_credit_card  ((3840312, 23),)   6.65%                 0
5          df_raw_installments  ((13605401, 8),)   0.01%                 0
6      df_raw_POS_CASH_balance  ((10001358, 8),)   0.07%                 0
7  df_raw_previous_application  ((1670214, 37),)  17.98%                 0
8     df_raw_sample_submission     ((48744, 2),)   0.00%                 0 

  column-wise information 
 df_raw_app_test 
                              dtype  # dupplicated entries avg_nan_col
SK_ID_CURR                    int64                    0.0       0.00%
NAME_CONTRACT_TYPE           object   

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,...,NaN,0.0514,NaN,NaN,NaN,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0


----------------------------------------------------------------------
 df_raw_app_train 
                              dtype  # dupplicated entries avg_nan_col
SK_ID_CURR                    int64                    0.0       0.00%
TARGET                        int64               307509.0       0.00%
NAME_CONTRACT_TYPE           object               307509.0       0.00%
CODE_GENDER                  object               307508.0       0.00%
FLAG_OWN_CAR                 object               307509.0       0.00%
...                             ...                    ...         ...
AMT_REQ_CREDIT_BUREAU_DAY   float64               307501.0      13.50%
AMT_REQ_CREDIT_BUREAU_WEEK  float64               307501.0      13.50%
AMT_REQ_CREDIT_BUREAU_MON   float64               307486.0      13.50%
AMT_REQ_CREDIT_BUREAU_QRT   float64               307499.0      13.50%
AMT_REQ_CREDIT_BUREAU_YEAR  float64               307485.0      13.50%

[122 rows x 3 columns]

first 3 rows



,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,...,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


----------------------------------------------------------------------
 df_raw_bureau_balance 
                 dtype  # dupplicated entries avg_nan_col
SK_ID_BUREAU     int64               26482530       0.00%
MONTHS_BALANCE   int64               27299828       0.00%
STATUS          object               27299917       0.00%

first 3 rows



...

----------------------------------------------------------------------
 df_raw_previous_application 
                               dtype  # dupplicated entries avg_nan_col
SK_ID_PREV                     int64                    0.0       0.00%
SK_ID_CURR                     int64              1331357.0       0.00%
NAME_CONTRACT_TYPE            object              1670210.0       0.00%
AMT_ANNUITY                  float64              1312254.0      22.29%
AMT_APPLICATION              float64              1576329.0       0.00%
AMT_CREDIT                   float64              1583410.0       0.00%
AMT_DOWN_PAYMENT             float64              1640935.0      53.64%
AMT_GOODS_PRICE              float64              1576328.0      23.08%
WEEKDAY_APPR_PROCESS_START    object              1670207.0       0.00%
HOUR_APPR_PROCESS_START        int64              1670190.0       0.00%
FLAG_LAST_APPL_PER_CONTRACT   object              1670212.0       0.00%
NFLAG_LAST_APPL_IN_DAY         int6

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0


----------------------------------------------------------------------
 df_raw_sample_submission 
              dtype  # dupplicated entries avg_nan_col
SK_ID_CURR    int64                    0.0       0.00%
TARGET      float64                48743.0       0.00%

first 3 rows



,SK_ID_CURR,TARGET
0,100001,0.5
1,100005,0.5
2,100013,0.5


----------------------------------------------------------------------
